In [ ]:
# In a Jupyter notebook cell, paste the following:

# 1) Switch to the interactive widget backend
%matplotlib widget

# 2) Imports
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from colour import SpectralDistribution, SDS_ILLUMINANTS, sd_to_XYZ, XYZ_to_sRGB
from scipy.interpolate import interp1d
import ipywidgets as widgets
from IPython.display import display
from nemo.analysis import Ensemble
from nemoview.visualization import FluorescentVialPlotter

# 4) Load your two Ensembles
s1 = Ensemble("../Tutorial/example_s1.lx")   # Molecule A
s2 = Ensemble("../Tutorial/npb_s1.lx")  # Molecule B

# fixed colors for each molecule
color_map = {'A':'tab:blue', 'B':'tab:orange'}

# 5) One-time figure & initial plot
fig, ax = plt.subplots(figsize=(8,4))
fig.subplots_adjust(right=0.6)  # make room for vials

ax.set_title("Fluorescence Emission and Vials")
ax.set_xlabel("Photon Energy (eV)")
ax.set_ylabel("Normalized Intensity")
ax.set_ylim(0, 1.1)

plotter = visualization.FluorescentVialPlotter(ax)

# helper to get emission for a given ensemble & dielectric
def get_xy(ens, a):
    emi = ens.emission((a, 1.0))
    return emi['Energy'].to_numpy(), emi['Diffrate'].to_numpy()

# initial data: none selected yet
lines = []  # will hold Line2D artists

# 6) Widgets: multi-select + slider
mol_select = widgets.SelectMultiple(
    options=[('Molecule A','A'), ('Molecule B','B')],
    value=['A', 'B'],  # start with A selected
    description='Molecules:'
)
a_slider = widgets.FloatSlider(
    value=1.0, min=1.0, max=10.0, step=0.1, description='Polarity (ε):'
)

# display widgets & figure
container = widgets.VBox([mol_select, a_slider, fig.canvas])
display(container)

# 7) Update callback
def update_plot(_):
    # clear old lines
    for ln in lines:
        ln.remove()
    lines.clear()
    # clear old vials
    plotter.clear()

    a = a_slider.value
    selected = mol_select.value  # tuple of 'A'/'B'
    for i, mol in enumerate(selected):
        ens = s1 if mol=='A' else s2
        x, y = get_xy(ens, a)
        #peak position
        peak = x[np.argmax(y)]
        # plot line (default color cycle)
        ln, = ax.plot(x, (y-y.min())/(y.max()-y.min()), label=f'Emission {mol}, Peak {peak:.2f} eV',color=color_map[mol])
        lines.append(ln)
        # add vial for this molecule
        plotter.add_vial(x, y, label=f'({mol})')

    # rescale axes
    ax.relim()
    ax.autoscale_view()
    ax.set_ylim(0, 1.1)
    # render all vials and legend
    plotter.render()
    alpha = (a -1)/(a+1)
    ax.legend(loc='upper left', bbox_to_anchor=(1.02, 1), borderaxespad=0, title=fr"ε={a:.2f} α={alpha:.3f}")
    fig.canvas.draw_idle()

# wire both widgets
mol_select.observe(update_plot, names='value')
a_slider.observe(update_plot, names='value')

# trigger initial draw
update_plot(None)
